In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
import datetime
%matplotlib inline

In [2]:
def max_stock(data):
    L = []
    for i in set(data["stock"]):
        counter = list(data["stock"].values).count(i)
        L.append(counter)                                        ###L为各股票出现的次数，即交易天数
    stock_id = list(set(data["stock"]))
    stock_m = stock_id[L.index(max(L))]                           ###选出交易天数最全的一只股票
    return stock_m

In [3]:
def best_perf(data, stock_eg, T, N1, N2, D):                                 ###data为原始数据，T为所要比较的涨跌幅的日期，N1，N2为T之前和T之后的时间跨度，K为最大成交总额及涨跌幅对比数

    data['stock'] = data['stock'].astype(str)                       ###更改stock的数据类型为字符型

    start = int(stock_eg[stock_eg['date'] ==  T].index.values-N1-1)-stock_eg.index[0]                  ###寻找T前N+1天的索引位置
    middle = int(stock_eg[stock_eg['date'] == T].index.values)-1-stock_eg.index[0]             ###寻找T日前一日的索引位置
    end = middle+N2                                                 ###统计涨跌幅最后一日的索引位置
    slc = stock_eg.iloc[[start, middle, middle+1, end],:]           ###选出所需计算的股票信息
    
    init = data[data['date'] == slc['date'].values[0]]
    middle = data[data['date'] == slc['date'].values[1]]
    determ = data[data['date'] == slc['date'].values[2]]
    fina = data[data['date'] == slc['date'].values[3]]
    secl_sum = init.append(middle).sort_index()
    secl_sum = secl_sum.append(determ).sort_index()
    secl_sum = secl_sum.append(fina).sort_index()                   ####将所有在T,T-N1-1,T-1，T-1+N2时有数据的股票信息合成一张表
    
    stock_intersect = list(set(init['stock']).intersection(set(middle['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(determ['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(fina['stock'])))        ###选出在上述四个时段都有数据的股票
    
    stock_f = pd.DataFrame(columns = ['date', 'stock', 'close', 'value', 'turnover_sum', 'turnover_N1', 'turnover_N2', 'change_N1','change_N2'])  ###建立一个空的dataframe
    
    for k in stock_intersect:
        starter = secl_sum[secl_sum['stock'] == k]
        starter = starter[['date', 'stock', 'close', 'value', 'turnover_sum']]
        turn_o = starter['turnover_sum']
        close = starter['close']
        differ_t = (turn_o.values[1]-turn_o.values[0])/turn_o.values[0]                 ###计算T前N1天的成交金额变化率
        differ_c0 = (close.values[1]-close.values[0])/close.values[0]              ###计算T前N1天的涨跌幅
        differ_t2 = (turn_o.values[3]-turn_o.values[1])/turn_o.values[1]                 ###计算T后N2天的成交金额变化率
        differ_c = (close.values[3]-close.values[1])/close.values[1]              ###计算T后N2天的涨跌幅
        stock_a = starter[starter['date'] == T]
        stock_a.insert(5,'turnover_N1',[differ_t/N1])
        stock_a.insert(6,'turnover_N2',[differ_t2/N2])
        stock_a.insert(7,'change_N1',[differ_c0/N1])                    ###插入N1天的平均日涨跌幅
        stock_a.insert(8,'change_N2',[differ_c/N2])                    ###插入N2天的平均日涨跌幅
        stock_f = stock_f.append(stock_a)
    stock_f = stock_f.sort_index()
    stock_f = stock_f[['date', 'stock', 'close', 'value', 'turnover_N1', 'turnover_N2', 'change_N1', 'change_N2']]
    
    ##stock_Vmin = stock_f.sort_values(by = ['value'], ascending = True).head(K)           ###取出前N1天市值最小的K个股票
    #stock_Tmax = stock_Vmin.sort_values(by = ['turnover_N1'], ascending = False).head(D)     ###取出前N1天成交金额最大的D个股票
    stock_Tmax = stock_f.sort_values(by = ['turnover_N1'], ascending = False).head(D)
    #stock_Cmax = stock_Tmax.sort_values(by = ['change_N1'], ascending = True).head(Z)         ###取出N1天平均日涨跌幅最小的Z个股票
    #stock_best = list(set(stock_Tmax['stock']).intersection(set(stock_Cmax['stock'])))   ###取出涨跌幅及成交金额都是最大K的股票     
    
    #stock_bestinf = pd.DataFrame(columns = ['date','stock','close','turnover_N1','turnover_N2','change_N1','change_N2'])
    #for i in stock_best:
       # infom = stock_Cmax[stock_Cmax['stock'] == i]
        #stock_bestinf = stock_bestinf.append(infom)                                ###获取这几只股票的具体信息
       # stock_bestinf = stock_bestinf.sort_index()
        #stock_bestinf = stock_bestinf[['stock', 'date', 'close', 'value', 'change_N1', 'change_N2', 'turnover_N1', 'turnover_N2']]
    stock_bestinf = stock_Tmax[['stock', 'date', 'close', 'value', 'change_N1', 'change_N2', 'turnover_N1', 'turnover_N2']]
    return stock_bestinf  

In [4]:
def turnover_fun(data, stock_eg, N1, N2, D):
    
    statis = pd.DataFrame(columns = ['date', 'avg_turnN1', 'avg_turnN2', 'avg_change1', 'avg_change2' ])

    for i in stock_eg['date'].values[N1+1:-N2+1]:
        try:
            stock_bestinf = best_perf(data, stock_eg, i, N1, N2, D)
            avg_turnN1 = stock_bestinf['turnover_N1'].mean()      ###求出满足条件股票的前N1日成交金额平均值
            avg_turnN2 = stock_bestinf['turnover_N2'].mean()      ###求出满足条件股票的后N2日成交金额平均值
            avg_change1 = stock_bestinf['change_N1'].mean()         ###求出满足条件股票的前N1日收益率平均值
            avg_change2 = stock_bestinf['change_N2'].mean()         ###求出满足条件股票的后N2日收益率平均值
            new_data = {'date':stock_bestinf['date'].values[0], 'avg_turnN1':avg_turnN1, 'avg_turnN2':avg_turnN2, 'avg_change1':avg_change1, 'avg_change2':avg_change2}
            new_add = pd.DataFrame(new_data, columns = ['date', 'avg_turnN1', 'avg_turnN2', 'avg_change1', 'avg_change2'],index=[0])     ###将列表合并
            statis = statis.append(new_add, ignore_index=True)
        except:
            continue

    return statis

In [5]:
data = pd.read_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据//2013_4/2013.csv',encoding='gbk',engine='python')

In [6]:
stock_m = max_stock(data)
#stock_m = 1
stock_eg = data[data['stock'] == stock_m]

In [7]:
statis = turnover_fun(data, stock_eg, 5, 20, 100)

In [8]:
statis

,date,avg_turnN1,avg_turnN2,avg_change1,avg_change2
0,2013-01-14,6.166367,0.138150,0.026640,0.002257
1,2013-01-15,2.876730,0.104054,0.032731,-0.000090
2,2013-01-16,1.479959,0.093480,0.028812,-0.000686
3,2013-01-17,3.248781,0.094099,0.030415,0.000542
4,2013-01-18,1.382030,0.085045,0.020268,0.001773
5,2013-01-21,0.678669,0.085486,0.026111,0.001398
6,2013-01-22,1.805999,0.084325,0.022948,0.000957
7,2013-01-23,1.267391,0.071952,0.013946,0.000658
8,2013-01-24,0.616383,0.071647,0.016193,0.000365
9,2013-01-25,0.398024,0.138415,0.016031,0.001547


In [9]:
statis.to_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据//2013_4/2013-5-20change.csv',index = False)